In [39]:
%matplotlib qt
import pyxdf
import mne
import numpy as np
from mne.datasets import misc
import os
import matplotlib.pyplot as plt
import matplotlib
import time
print(os.getcwd())

fnames = ["sub-elpida_ses-16_10_23_task-ASSR3_acq-ultracortex_run-001_eeg.xdf"]
raws = []
for fnameIndex in range(len(fnames)): 
    streams, header = pyxdf.load_xdf(fnames[fnameIndex])
    # get sampling rate
    sfreq = float(streams[1]["info"]["nominal_srate"][0])

    # get data
    data = streams[1]["time_series"].T
    dataTimestamps = streams[1]["time_stamps"].T
    # get markers
    markers = streams[0]["time_series"].T
    markerTimestamps = streams[0]["time_stamps"].T

    #in case of there is a switch between stim channel and data channels
    if  data.shape[0] != 8:
        data = streams[0]["time_series"].T
        dataTimestamps = streams[0]["time_stamps"].T
        sfreq = float(streams[0]["info"]["nominal_srate"][0])
        markers = streams[1]["time_series"].T
        markerTimestamps = streams[1]["time_stamps"].T
        assert data.shape[0] == 8,  fnames[fnameIndex] + " has different channel amounts" # check that we have what we think we have (8 eeg channels)

    

    # get together the event channel with the data, make a "stim 014" type channel as annotation: time, duration, event type 
    # add markers: What needs to be done: we have markers as several events, and their timestamp. then, we have the data
    # and their timestamps, to the raw object we should add one more channel, initially full of zeros with the length of the measurement
    
    
    # This is because microvolts are being measured, but mne plot expects values in volts.
    markers = markers[0]
    data *= 1e-6 # uV -> V and preamp gain. IDK what will preamp gain do. (multiply / divide by gain? 24?)

    markerArr = np.zeros_like(data[0])
    
    for markerid in range(len(markers)):
        absolute_differences = np.abs(dataTimestamps - markerTimestamps[markerid])
        closest_index = np.argmin(absolute_differences)
        markerArr[closest_index] = markers[markerid]

    # TEST: this gives the amount of events, and also their index. divided by the sampling freq, you should see the time differences of
    #each part of the experiment. 
    nonzero_indices = np.nonzero(markerArr)
    num_nonzero = len(nonzero_indices[0])
    print("uniques", np.unique(markerArr))
    print("Number of nonzero elements:", num_nonzero)
    print("Indices of nonzero elements:", nonzero_indices[0] / sfreq)

    #add marker arr to data
    combined = np.c_[data.T,markerArr].T

    # create info object set channel names
    info = mne.create_info(["T6", "T4", "F8", "C4", "C3", "T5", "T3", "F7", "stim"], sfreq, ["eeg", "eeg","eeg", "eeg", "eeg", "eeg", "eeg", "eeg", "stim"])
    raw = mne.io.RawArray(combined, info)
    raw.set_montage("standard_1020")

    #preliminary data analyzis: (fancy term for plotting it and looking at it). Task: review data and select a consistently bad channel for exclusion
    print("Task: review data and select a consistently bad channel for exclusion.")
    
    raw.plot(title=fnames[fnameIndex] + ", Task: review data and select consistently bad channels", duration=1)
    raw.compute_psd().plot()
    plt.title(fnames[fnameIndex])
    plt.show()
    raws.append((raw, fnames[fnameIndex]))

C:\Users\elpid\Downloads\thesis\ThesisRepository\scripts
uniques [0. 1. 2.]
Number of nonzero elements: 30
Indices of nonzero elements: [  10.832   20.764   80.92    90.856  151.016  160.944  221.112  231.036
  291.196  301.128  366.296  376.22   436.384  446.308  506.476  516.396
  576.556  586.488  646.648  656.58   721.744  731.668  791.844  801.764
  861.92   871.852  932.02   941.944 1002.108 1012.032]
Creating RawArray with float64 data, n_channels=9, n_times=272970
    Range : 0 ... 272969 =      0.000 ...  1091.876 secs
Ready.
Task: review data and select a consistently bad channel for exclusion.
Effective window size : 1.024 (s)
Channels marked as bad:
none


In [50]:
epochs = []
for rawpack in raws: 
    raw = rawpack[0]
    fname = rawpack[1]
    low_cut = 0.1
    high_cut = 50
    filtered_data = raw.copy().filter(low_cut,high_cut,method='iir')
    notch_freqs = [ 50, 100]
    filtered_data = filtered_data.notch_filter(freqs=notch_freqs, filter_length='auto', method='fir' ,notch_widths=4 , picks="eeg")

    #get events    
    events = mne.find_events(filtered_data, stim_channel="stim")
    #events: 1: start rest, 2: start assr, 3: artifact
    #setup values above which you reject (peak to peak)
    reject_criteria = dict(eeg=20000e-6)

    #set flat criteria (epochs where nothing happens really (min peak to peak))
    flat_criteria = dict(eeg=1e-6)

    #make epochs
    stimEpochs = mne.Epochs(filtered_data, events, event_id=2, tmin=-10, tmax=60, baseline=(None,0), preload=True, reject=reject_criteria, flat=flat_criteria)
    stimEpochs.plot_drop_log()
    epochs.append((stimEpochs, fnames[fnameIndex]))

    stimEpochs.plot(title=fname+"filtered")
    stimEpochs.compute_psd().plot()
    plt.title(fname)
    plt.show()

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 50 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.10, 50.00 Hz: -6.02, -6.02 dB

Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 1651 samples (6.604 sec)

30 events found
Event IDs: [1 2]
Not setting metadata
15 matching events found
Setting baseline interval to [-10.0, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 15 events and 17501 original time points ...
0 bad epochs dropped
Not setting m

In [49]:
#filter_params = mne.filter.create_filter(raw.get_data(),raw.info['sfreq'],l_freq=0.1,h_freq=50,method='iir')
#mne.viz.plot_filter(filter_params,raw.info['sfreq'],flim=(0.01,raw.info['sfreq']))